4 MODEL DEPLOYMENT

So far there is a module for getting and storing the data. There is code to train the model and clean its predictions. Put all this pieces  together and deploy the model with an API that others can use to train their own models and predict volatility. First start by creating a model for all the code created in the last lesson. Then complete the main module, which will hold the FastAPI application with two paths: one for model training and one for prediction.


In [14]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob
import warnings

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
warnings.simplefilter(action="ignore", category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


4.1 Model Module

Create a model module to make the code  reusable
Start by instantiating a repository to be used for testing the module as they are being built

In [15]:
#Create a SQLRepository named repo and attach it to a SQLite connection.
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In the model module, create a definition for a GarchModel model class. Use __init__ method. 

In [16]:
#Test your class using the assert statements
from model import GarchModel

# Instantiate a `GarchModel`
gm_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_cic.ticker == "CIC"
assert gm_cic.repo == repo
assert not gm_cic.use_new_data
assert gm_cic.model_directory == settings.model_directory

Turn the wrangle_data function in to a method for the GarchModel class.


In [17]:
#Use the assert statements below to test the method by getting and wrangling data for Co-operative  Bank

# Instantiate `GarchModel`, use new data
model_coop = GarchModel(ticker="COOP", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_coop, "data")

# Wrangle data
model_coop.wrangle_data(n_observations=1300)

# Does model now have `data` attribute?
assert hasattr(model_coop, "data")

# Is the `data` a Series?
assert isinstance(model_coop.data, pd.Series)

# Is Series correct shape?
assert model_coop.data.shape == (1300,)

model_coop.data.head()

date
2017-12-29     5.899227
2018-01-02     0.482864
2018-01-03    -2.461322
2018-01-04    -2.162942
2018-01-05    11.520511
Name: return, dtype: float64

Use the previous code for creating a Garch model to create a fit method for the GarchModel class

In [18]:
#Use the assert statements below to test the method

# Instantiate `GarchModel`, use old data
model_coop = GarchModel(ticker="COOP", repo=repo, use_new_data=False)

# Wrangle data
model_coop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_coop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_coop, "model")

# Is model correct data type?
assert isinstance(model_coop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_coop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_coop.model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2418.18
Distribution:                  Normal   AIC:                           4844.36
Method:            Maximum Likelihood   BIC:                           4863.99
                                        No. Observations:                 1000
Date:                Thu, Mar 02 2023   Df Residuals:                      999
Time:                        03:52:23   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1654  8.077e-02      2.048  4.061e-02 [7.072e-03,  0.324]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2083      0.165      1.261      0.207   [ -0.115,  0.532]
alpha[1]       0.1052  4.697e-02      2.240  2.512e-02 [1.313e-02,  0.197]
beta[1]        0.8762  5.134e-02     17.069  2.537e-65   [  0.776,  0.977]
==========================================================================

Covariance estimator: robust
"""

Create a predict_volatility method for the GarchModel class using the code from the garch-models module add the clean_prediction function as a helper method to clean the dictionary

In [19]:
# Generate prediction from `model_coop`
prediction = model_coop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-03-02T00:00:00': 1.9023773750112967,
 '2023-03-03T00:00:00': 1.939101491114852,
 '2023-03-06T00:00:00': 1.9744799057687676,
 '2023-03-07T00:00:00': 2.0085958278144056,
 '2023-03-08T00:00:00': 2.041524234621052}

Add two more methods to the GarchModel to save a trained model and load it up when needed

In [20]:
# Save `model_coop` model, assign filename
filename = model_coop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_coop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models/2023-03-02T03:52:23.627654_COOP.pkl'

In [21]:
#Create a load function below that will take a ticker symbol as input and return a model. 
def load(ticker):

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")


    # Try to find path of latest model
    try:
        model_path = model_path=sorted(glob(pattern))[-1]
   
    # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No model trained for '{ticker}'.")


    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

In [22]:
#Load  model_coop 
# Assign load output to `model`
model_coop = load(ticker="COOP")

# Does function return an `ARCHModelResult`
assert isinstance(model_coop, ARCHModelResult)

# Check model parameters
model_coop.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2418.18
Distribution:                  Normal   AIC:                           4844.36
Method:            Maximum Likelihood   BIC:                           4863.99
                                        No. Observations:                 1000
Date:                Thu, Mar 02 2023   Df Residuals:                      999
Time:                        03:52:23   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1654  8.077e-02      2.048  4.061e-02 [7.072e-03,  0.324]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2083      0.165      1.261      0.207   [ -0.115,  0.532]
alpha[1]       0.1052  4.697e-02      2.240  2.512e-02 [1.313e-02,  0.197]
beta[1]        0.8762  5.134e-02     17.069  2.537e-65   [  0.776,  0.977]
==========================================================================

Covariance estimator: robust
"""

In [23]:
 #Transform your load function into a method for the GarchModel class then test the method using assert statements

model_coop = GarchModel(ticker="COOP", repo=repo, use_new_data=False)

# Check that new `model_coop_test` doesn't have model attached
assert not hasattr(model_coop, "model")

# Load model
model_coop.load()

# Does `model_coop_test` have model attached?
assert hasattr(model_coop, "model")

model_coop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2418.18
Distribution:                  Normal   AIC:                           4844.36
Method:            Maximum Likelihood   BIC:                           4863.99
                                        No. Observations:                 1000
Date:                Thu, Mar 02 2023   Df Residuals:                      999
Time:                        03:52:23   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1654  8.077e-02      2.048  4.061e-02 [7.072e-03,  0.324]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.2083      0.165      1.261      0.207   [ -0.115,  0.532]
alpha[1]       0.1052  4.697e-02      2.240  2.512e-02 [1.313e-02,  0.197]
beta[1]        0.8762  5.134e-02     17.069  2.537e-65   [  0.776,  0.977]
==========================================================================

Covariance estimator: robust
"""

This marks the end of the model module

4.2. Main Module

First step, create an app object

The app will be FastAPI application

In the main module, instantiate a FastAPI application named app.

 The "/fit" Path

The first path will allow the user to fit a model to stock data when they make a post request to our server. They'll have the choice to use new data from AlphaVantage, or older data that's already in our database. When a user makes a request, they'll receive a response telling them if the operation was successful or whether there was an error.Use the pydantic library to check that each request has the correct parameters and data types by using special data classes that we need to define. Our "/fit" path will take user input and then output a response, so we need two classes: one for input and one for output.

 Create definitions for a FitIn and a FitOut data class. The FitIn class inherits from the pydantic BaseClass, and the FitOut class inherits from the FitIn class

In [24]:
#Experimenting with the FitIn and FitOut classes
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker="EQTY",
    use_new_data= True,
    n_observations= 2000,
    p=1,
    q=1
)

print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker="EQTY",
    use_new_data= True,
    n_observations= 2000,
    p=1,
    q=1,
    success= True,
    message="Method is ready to rock!"
)
print(fo)

ticker='EQTY' use_new_data=True n_observations=2000 p=1 q=1
ticker='EQTY' use_new_data=True n_observations=2000 p=1 q=1 success=True message='Method is ready to rock!'


In [25]:
#Create a build_model function in the main module
from main import build_model

# Instantiate `GarchModel` with function
model_cic = build_model(ticker="CIC", use_new_data=False)

# Is `SQLRepository` attached to `model_cic`?
assert isinstance(model_cic.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_cic.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_cic.ticker == "CIC"

# Is `use_new_data` attribute correct?
assert not model_cic.use_new_data

model_cic

Create a "/fit" path for the app. It will take a FitIn object as input, and then build a GarchModel using the build_model function. The model will wrangle the needed data, fit to the data, and save the completed model. Finally, it will send a response in the form of a FitOut object. while at the same time handle any errors that may arise using the exceptions set and the pydantic Base Class

Create a post request to hit the "/fit" path running at "http://localhost:8008". Train a GARCH(1,1) model on 2000 observations of the CIC data already downloaded. Pass in the parameters as a dictionary using the json argument.

In [26]:
# URL of `/fit` path
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker" : "CIC",
    "use_new_data": True,
    "n_observations": 2000,
    "p":1,
    "q":1
    
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'CIC',
 'use_new_data': True,
 'n_observations': 2000,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "trained and saved 'models/2023-03-02T03:52:25.191594_CIC.pkl'.Metrics: AIC 5201.2553, BIC 5222.0956."}

Now we can train models using the API we created.

The "/predict" Path

For the "/predict" path, users will be able to make a post request with the ticker symbol they want a prediction for and the number of days they want to forecast into the future. The app will return a forecast or, if there's an error, a message explaining the problem.

Create definitions for the PredictIn and PredictOut data class. The PredictIn class inherits from the pydantic BaseModel, and the PredictOut class inherits from the PredictIn class. test your classes.

In [27]:
#Test the classes
from main import PredictIn, PredictOut

pi = PredictIn(ticker="DTK", n_days=5)
print(pi)

po = PredictOut(
    ticker="DTK", n_days=5, success=True, forecast={}, message="success"
)
print(po)

ticker='DTK' n_days=5
ticker='DTK' n_days=5 success=True forecast={} message='success'


Create a "/predict" path for the app. It takes a PredictIn object as input, build a GarchModel, load the most recent trained model for the given ticker, and generate a dictionary of predictions. It then returns a PredictOut object with the predictions included. It includes Exceptions to handle any errors that may arise.

Create a post request to hit the "/predict" path running at "http://localhost:8008". Get the 5-day volatility forecast for CIC. 

In [28]:
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {"ticker": "CIC","n_days":3}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

{'ticker': 'CIC',
 'n_days': 3,
 'success': True,
 'forecast': {'2023-03-02T00:00:00': 1.7804907255485452,
  '2023-03-03T00:00:00': 1.7808601360578307,
  '2023-03-06T00:00:00': 1.7812294691345432},
 'message': 'Done'}